In [23]:
# Importamos librerías

# Librerías para manejo de datos
import pandas as pd

# Librerías para operaciones aleatorias
from random import random

# Librerías para anotaciones de tipo y futuras versiones de Python
from __future__ import annotations

# Librerías para anotaciones de tipo
from typing import Dict, List

# Librerías para imprimir tablas
from tabulate import tabulate

Importamos nuestros datos.

In [24]:
partidos = pd.read_csv('../../data/partidos_limpio.csv')

Creamos una clase que nos permite definir y manipular un grafo que representa las transiciones entre diferentes estados. Cada estado en el grafo representa una situación o condición, y las transiciones entre estados están definidas por probabilidades. Por ejemplo, si estamos modelando el comportamiento de un equipo deportivo, los estados podrían ser "ganar", "empatar" o "perder", y las transiciones entre estos estados podrían estar determinadas por la probabilidad de que el equipo gane, empate o pierda en un partido determinado.

In [25]:
class MarkovChainGraph:
    """
    Grafo para ejecutar el Algoritmo de Cadena de Markov
    """

    def __init__(self):
        self.transitions: Dict[str, Dict[str, float]] = {}

    def add_transition_probability(
        self, from_state: str, to_state: str, probability: float
    ) -> None:
        if from_state not in self.transitions:
            self.transitions[from_state] = {}
        self.transitions[from_state][to_state] = probability

    def get_states(self) -> List[str]:
        return list(self.transitions.keys())

    def transition(self, current_state: str) -> str:
        transition_probabilities = self.transitions.get(current_state, {})
        total_probability = sum(transition_probabilities.values())
        if total_probability == 0:
            return ""
        random_value = random()
        cumulative_probability = 0
        for next_state, probability in transition_probabilities.items():
            cumulative_probability += probability / total_probability
            if random_value <= cumulative_probability:
                return next_state
        return ""

Ahora creamos una función que calcule automáticamente la matriz de transición a partir de los datos de los partidos contenidos en un DataFrame. Esta matriz de transición es esencial para utilizar el algoritmo de Markov de manera efectiva, ya que define las probabilidades de transición entre diferentes estados.

In [26]:
def get_transition_matrix_from_dataframe(
    df: pd.DataFrame, home_col: str, away_col: str, result_col: str
) -> Dict[str, Dict[str, float]]:
    """
    Calcula la matriz de transición de una cadena de Markov basada en los resultados de los partidos de un DataFrame.

    Args:
    df (DataFrame): DataFrame que contiene los resultados de los partidos.
    home_col (str): Nombre de la columna para los equipos locales.
    away_col (str): Nombre de la columna para los equipos visitantes.
    result_col (str): Nombre de la columna para los resultados de los partidos.

    Returns:
    dict: Matriz de transición representada como un diccionario.
    """
    transitions = []

    # Convertir los resultados de los partidos en estados ('win', 'draw', 'lose')
    result_map = {'H': 'win', 'D': 'draw', 'A': 'lose'}
    df['from_state'] = df[result_col].map(result_map)

    # Calcular la siguiente transición como el estado siguiente después del partido
    df['to_state'] = df.groupby(home_col)['from_state'].shift(-1)

    # Eliminar filas con NaN resultantes de la última transición de cada equipo
    df.dropna(subset=['to_state'], inplace=True)

    # Contar ocurrencias de transición
    transition_counts = pd.crosstab(df['from_state'], df['to_state'])

    # Normalizar frecuencias para obtener probabilidades
    transition_probabilities = transition_counts.div(transition_counts.sum(axis=1), axis=0)

    # Convertir probabilidades de transición a un diccionario
    transition_matrix = transition_probabilities.to_dict(orient='index')

    return transition_matrix

Podemos obtener una matriz de transición que nos muestra las probabilidades de los resultados de cualquier partido. Es decir, viendo el historial del torneo, vemos las probabilidades (da igual el equipo) gane, pierda o empate un partido. 

Sin embargo, es mucho más práctico ver la matriz de transición de un equipo. Por lo tanto, haremos una función que nos filtre los partidos del equipo del que queramos obtener la matriz de transición.

In [27]:
def partidos_equipo(equipo, df):
    """
    Filtra los partidos en los que un equipo específico estuvo involucrado.

    Args:
    equipo (str): Nombre del equipo.
    df (DataFrame): DataFrame que contiene los datos de los partidos.

    Returns:
    DataFrame: DataFrame filtrado que contiene los partidos del equipo especificado.
    """
    return df[(df['Home'] == equipo) | (df['Away'] == equipo)].copy()

Ahora pongamos todo este código en funcionamiento. 

Primero filtremos los partidos de los equipos de los que queramos obtener su matriz de transición. Por ejemplo, podemos ver los partidos de los equipos que pasaron a la semifinal del torneo.

In [28]:
# Filtrar los partidos de los cuatro equipos de semifinales
partidos_real_madrid = partidos_equipo('Real Madrid', partidos)
partidos_bayer_munich = partidos_equipo('Bayern Munich', partidos)
partidos_psg = partidos_equipo('Paris S-G', partidos)
partidos_dortmund = partidos_equipo('Dortmund', partidos)


Ahora obtenemos las matrices de transición de cada equipo.

In [29]:
# Obtener la matriz de transición de los resultados de los partidos del Real Madrid
matriz_transicion = get_transition_matrix_from_dataframe(partidos_real_madrid, 'Home', 'Away', 'Results')

# Mostrar la matriz de transición con tabulate
headers = ["", "draw", "lose", "win"]
rows = []

# Convertir la matriz de transición en una lista de listas para tabulate
for estado, transiciones in matriz_transicion.items():
    transiciones_str = [f"{valor:.2f}" for valor in transiciones.values()]
    rows.append([estado] + transiciones_str)

# Imprimir la tabla con tabulate
print("Matriz de Transición del Real Madrid:")
print(tabulate(rows, headers=headers, tablefmt="pretty"))


Matriz de Transición del Real Madrid:
+------+------+------+------+
|      | draw | lose | win  |
+------+------+------+------+
| draw | 0.08 | 0.25 | 0.67 |
| lose | 0.15 | 0.40 | 0.45 |
| win  | 0.21 | 0.13 | 0.66 |
+------+------+------+------+


In [30]:
# Obtener la matriz de transición de los resultados de los partidos del Bayern Munich
matriz_transicion = get_transition_matrix_from_dataframe(partidos_bayer_munich, 'Home', 'Away', 'Results')

# Mostrar la matriz de transición con tabulate
rows = []

# Convertir la matriz de transición en una lista de listas para tabulate
for estado, transiciones in matriz_transicion.items():
    transiciones_str = [f"{valor:.2f}" for valor in transiciones.values()]
    rows.append([estado] + transiciones_str)

# Imprimir la tabla con tabulate
print("Matriz de Transición del Bayern Munich:")
print(tabulate(rows, headers=headers, tablefmt="pretty"))

Matriz de Transición del Bayern Munich:
+------+------+------+------+
|      | draw | lose | win  |
+------+------+------+------+
| draw | 0.00 | 0.33 | 0.67 |
| lose | 0.06 | 0.33 | 0.61 |
| win  | 0.21 | 0.24 | 0.55 |
+------+------+------+------+


In [31]:
# Obtener la matriz de transición de los resultados de los partidos del Dortmund
matriz_transicion = get_transition_matrix_from_dataframe(partidos_dortmund, 'Home', 'Away', 'Results')

# Mostrar la matriz de transición con tabulate
rows = []

# Convertir la matriz de transición en una lista de listas para tabulate
for estado, transiciones in matriz_transicion.items():
    transiciones_str = [f"{valor:.2f}" for valor in transiciones.values()]
    rows.append([estado] + transiciones_str)

# Imprimir la tabla con tabulate
print("Matriz de Transición del Dortmund:")
print(tabulate(rows, headers=headers, tablefmt="pretty"))

Matriz de Transición del Dortmund:
+------+------+------+------+
|      | draw | lose | win  |
+------+------+------+------+
| draw | 0.00 | 1.00 | 0.00 |
| lose | 0.00 | 0.40 | 0.60 |
| win  | 0.10 | 0.30 | 0.60 |
+------+------+------+------+


In [32]:
# Obtener la matriz de transición de los resultados de los partidos del Paris S-G
matriz_transicion = get_transition_matrix_from_dataframe(partidos_psg, 'Home', 'Away', 'Results')

# Mostrar la matriz de transición con tabulate
rows = []

# Convertir la matriz de transición en una lista de listas para tabulate
for estado, transiciones in matriz_transicion.items():
    transiciones_str = [f"{valor:.2f}" for valor in transiciones.values()]
    rows.append([estado] + transiciones_str)

# Imprimir la tabla con tabulate
print("Matriz de Transición del Paris S-G:")
print(tabulate(rows, headers=headers, tablefmt="pretty"))

Matriz de Transición del Paris S-G:
+------+------+------+------+
|      | draw | lose | win  |
+------+------+------+------+
| draw | 0.40 | 0.40 | 0.20 |
| lose | 0.09 | 0.45 | 0.45 |
| win  | 0.33 | 0.25 | 0.42 |
+------+------+------+------+
